In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("/home/pranjal/Downloads/custom_neural_network_library/housing_train.csv")

In [3]:
def df_column_switch(df, column1, column2):
    i = list(df.columns)
    a, b = i.index(column1), i.index(column2)
    i[b], i[a] = i[a], i[b]
    df = df[i]
    return df

df = df_column_switch(df, "longitude", "median_house_value")

In [4]:
data = np.array(df)
m,n = data.shape
np.random.shuffle(data)
print(f"m:{m}")
print(f"n:{n}")

m:17000
n:9


In [5]:
val_data = data[0 : 3000].T
Y_val = val_data[0]
X_val = val_data[1 : n]
# X_val = X_val / 255.

train_data = data[3000 : m].T
Y_train = train_data[0]
X_train = train_data[1 : n]
# X_train = X_train / 255.

In [6]:
print(X_train.shape)
print(X_val.shape)
print(Y_train.shape)
print(Y_val.shape)

(8, 14000)
(8, 3000)
(14000,)
(3000,)


In [7]:
def parameterss(hidden_layers, n_neurons, n_classes):
        weights = {}
        biases = {}

        for i in range(hidden_layers):

            if i == 0:
                #For the first hidden layer
                weights[f"W{i + 1}"] = np.random.rand(n_neurons, 8) - 0.5
                biases[f"b{i + 1}"] = np.random.rand(n_neurons, 1) - 0.5
            else:
                # For rest of the hidden layers
                weights[f"W{i + 1}"] = np.random.rand(n_neurons, n_neurons) - 0.5
                biases[f"b{i + 1}"] = np.random.rand(n_neurons, 1) - 0.5

        #For final output layer
        weights[f"W{hidden_layers + 1}"] = np.random.rand(n_classes, n_neurons) - 0.5
        biases[f"b{hidden_layers + 1}"] = np.random.rand(n_classes, 1) - 0.5

        return weights, biases

In [8]:
w, b = parameterss(2,300,1)

In [9]:
w.keys()

dict_keys(['W1', 'W2', 'W3'])

In [10]:
b.keys()

dict_keys(['b1', 'b2', 'b3'])

In [11]:
w['W1']

array([[-0.18611549, -0.26321804, -0.32789059, ...,  0.28341736,
        -0.31016138, -0.39242029],
       [ 0.38437684,  0.45175106,  0.02364249, ...,  0.18276074,
        -0.35287573, -0.38529028],
       [ 0.3943126 , -0.02043891,  0.1958145 , ...,  0.44756814,
        -0.26011882, -0.27570609],
       ...,
       [-0.29415629, -0.44589771,  0.17435828, ..., -0.45680007,
        -0.42203831,  0.46916965],
       [ 0.12351521,  0.37402411, -0.05652371, ..., -0.12008626,
        -0.44333486,  0.38958878],
       [ 0.41571924,  0.37077353, -0.02620328, ..., -0.29266299,
        -0.0162827 , -0.28536391]])

In [12]:
def ReLU(Z):
    return np.maximum(Z ,0)
    
def Linear(Z):
    return Z   

def derivative_ReLU(Z):
    return Z > 0

def derivative_linear(Z):
    return 1

In [13]:
def forward_propagation(weights, biases, X):
    l = len(weights)
    p = {}

    for i in range(1, l+1):
        if i == 1 :
            p[f"Z{i}"] = weights['W1'].dot(X) + biases['b1']
            p[f"A{i}"] = ReLU( weights['W1'].dot(X) + biases['b1'])
        elif i > 1 and i < l:
            p[f"Z{i}"] = weights[f"W{i}"].dot(p[f"A{i-1}"]) + biases[f"b{i}"]
            p[f"A{i}"] = ReLU(weights[f"W{i}"].dot(p[f"A{i-1}"]) + biases[f"b{i}"])
        elif i == l:
            p[f"Z{i}"] = weights[f"W{i}"].dot(p[f"A{i-1}"]) + biases[f"b{i}"]
            p[f"A{i}"] = Linear(weights[f"W{i}"].dot(p[f"A{i-1}"]) + biases[f"b{i}"])

    return p

In [14]:
a = forward_propagation(w, b, X_train)

In [15]:
a.keys()

dict_keys(['Z1', 'A1', 'Z2', 'A2', 'Z3', 'A3'])

In [16]:
def back_propagation(a, weights, X_train, Y_train):
    p = {}
    l = len(weights)

    for i in range(l, -1+1, -1):
        if i == l:
            p[f"dZ{i}"] = a[f"A{i}"] - Y_train
            p[f"dW{i}"] = 1 / m * p[f"dZ{i}"].dot(a[f"A{i-1}"].T)
            p[f"db{i}"] = 1 / m * np.sum(p[f"dZ{i}"], axis=1, keepdims=True)
        elif i < l and i > 1:
            p[f"dZ{i}"] = weights[f"W{i+1}"].T.dot(p[f"dZ{i+1}"]) * derivative_ReLU(a[f"Z{i}"])
            p[f"dW{i}"] = 1 / m * p[f"dZ{i}"].dot(a[f"A{i-1}"].T)
            p[f"db{i}"] = 1 / m * np.sum(p[f"dZ{i}"], axis=1, keepdims=True)
        elif i == 1:
            p[f"dZ{i}"] = weights[f"W{i+1}"].T.dot(p[f"dZ{i+1}"]) * derivative_linear(a[f"Z{i}"])
            p[f"dW{i}"] = 1 / m * p[f"dZ{i}"].dot(X_train.T)
            p[f"db{i}"] = 1 / m * np.sum(p[f"dZ{i}"], axis=1, keepdims=True)

    return  p

In [17]:
s = back_propagation(a, w, X_train, Y_train)

In [18]:
# l = 3
# for i in range(l,-1+1,-1):
#     print(i)

In [19]:
s.keys()

dict_keys(['dZ3', 'dW3', 'db3', 'dZ2', 'dW2', 'db2', 'dZ1', 'dW1', 'db1'])

In [20]:
def update_parameters(weights, biases, s, alpha):
    p1 = {}
    p2 = {}

    l = len(weights)

    for i in range(1, l + 1):
        p1[f"W{i}"] = weights[f"W{i}"] - alpha * s[f"dW{i}"]
        p2[f"b{i}"] = biases[f"b{i}"] - alpha * s[f"db{i}"]

    return p1, p2

In [21]:
w, b = update_parameters(w, b, s, 0.1)

In [22]:
w.keys()

dict_keys(['W1', 'W2', 'W3'])

In [23]:
b.keys()

dict_keys(['b1', 'b2', 'b3'])

In [24]:
w['W1']

array([[-8.36157060e+05, -6.95920801e+05, -6.51724205e+07, ...,
        -1.15735148e+07, -1.15437232e+05,  2.82118693e+06],
       [-2.74315471e+05, -2.32544066e+05, -2.05483369e+07, ...,
        -3.72852641e+06, -3.67849326e+04,  9.25662721e+05],
       [ 3.30943800e+05,  2.77799282e+05,  2.52136586e+07, ...,
         4.61455710e+06,  4.45789048e+04, -1.11635449e+06],
       ...,
       [ 1.18364295e+06,  9.85004536e+05,  9.10004702e+07, ...,
         1.67040671e+07,  1.55242393e+05, -3.99310686e+06],
       [ 1.14163283e+06,  9.51259858e+05,  8.71433567e+07, ...,
         1.61487898e+07,  1.47875188e+05, -3.85461048e+06],
       [-5.23039749e+05, -4.36869584e+05, -3.91268673e+07, ...,
        -6.98111182e+06, -7.16788987e+04,  1.76339908e+06]])

In [25]:
print(a['A3'])
print(a['A3'].shape)

[[16177.87939431 14245.15933601 32602.43985297 ... 34358.02927606
  25472.21986107  7560.1677022 ]]
(1, 14000)


In [26]:
print(Y_train)
print(Y_train.shape)

[171100. 113900. 267500. ... 300000. 342000. 204500.]
(14000,)


In [32]:
def root_mean_squared_error(Y_predicted, Y):
    mse = np.mean((Y_predicted - Y)**2)
    rmse = np.sqrt(mse)
    return rmse

In [33]:
last_element_key, last_element_value = list(a.items())[-1]
print(last_element_key)

A3


In [34]:
prediction = root_mean_squared_error(last_element_value, Y_train)

In [35]:
prediction

224957.47544948166